In [1]:
import os
import pandas as pd

In [2]:
path = 'C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\climate_data'

In [5]:
# Define as conversões de tipos desejadas
				
conversoes = {
        'Data': 'datetime64[ns]',
        'Hora UTC': 'string',    
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'float',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'float',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'float',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'float',
    }

In [3]:
df_raw_clima = pd.read_parquet('C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\df_raw_clima.parquet',use_pandas_metadata=False, engine="pyarrow")

In [1]:
from preprocessing.data_cleaning import DataCleaner
import pandas as pd

df_raw_clima = pd.read_parquet('C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\raw\\df_raw_clima.parquet',use_pandas_metadata=False, engine="pyarrow")

conversoes_ = {
        'Data': 'datetime64[ns]',
        'Hora UTC': 'string',    
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'float',
        'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'float',
        'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'float',
        'VENTO, VELOCIDADE HORARIA (m/s)': 'float',
    }

cleaner = DataCleaner(df_raw_clima)

df_trusted_clima = (
    cleaner
    .tratar_divisor_milhar('TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)')
    .tratar_divisor_milhar('TEMPERATURA DO PONTO DE ORVALHO (°C)')
    .tratar_decimal('VENTO, VELOCIDADE HORARIA (m/s)')
    .tratar_decimal('PRECIPITAÇÃO TOTAL, HORÁRIO (mm)')
    .tratar_nulos("all",0)
    .ajustar_tipos_colunas(conversoes_)
    .get_df()
    )

# Salva o DataFrame limpo
#df_trusted_clima.to_parquet('data/trusted/df_trusted_clima.parquet', engine='pyarrow', index=False)

In [39]:
df_clima = df_clima.rename(columns={'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)':'precipitacao_total',
                                    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)':'temp_media_bulbo_seco',
                                    'TEMPERATURA DO PONTO DE ORVALHO (°C)':'temp_media_ponto_orvalho',
                                    'VENTO, VELOCIDADE HORARIA (m/s)':'velocidade_media_vento',})

In [ ]:
df_clima.groupby(['Data']).agg({'precipitacao_total': 'mean',
                                'temp_media_bulbo_seco': 'mean',
                                'temp_media_ponto_orvalho': 'sum',
                                'velocidade_media_vento': 'mean'}).reset_index()